# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

In [1]:
!pip install -qU langchain==0.2.7 langchain-openai langchain-cohere rank_bm25 langchain_core==0.2.40

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [5]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [9]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget


### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [11]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 23, 20, 32, 14, 455720)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [7]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [8]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [11]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [17]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The movie was praised for its slickness, brilliant action sequences, Keanu Reeves' performance, and overall entertainment value."

In [18]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\'.'

In [19]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The story is filled with violent action, shootouts, and breathtaking fights.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [12]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [13]:
naive_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [22]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Opinions on John Wick seem to vary. Some people really enjoyed it for its action and style, while others found it lacking in plot and substance. It ultimately depends on personal preferences.'

In [23]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I'm sorry, there are no reviews with a rating of 10 in the provided context."

In [24]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is an action movie series starring Keanu Reeves. It features intense action scenes, emotional depth, and impressive choreography. The series follows the story of John Wick, a retired hitman seeking vengeance. Each movie in the series builds upon the previous one, showcasing more fights, assassins, and challenges for the main character. Overall, John Wick is known for its brutal action and engaging storyline.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [14]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Let's create our chain again, and see how this does!

In [15]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [27]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick. The reviews mention that it was a remarkable, surprising film that caught viewers off guard, with slick, violent fun and elaborate action sequences that set it apart from other action films.'

In [28]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". The URL to that review is: \'/review/rw4854296/?ref_=tt_urv\'.'

In [29]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, after resolving his issues with the Russian mafia, John Wick is forced back into action when mobster Santino D'Antonio asks him to kill his sister in Rome in order to gain power within criminal organizations. Wick completes the task but then becomes the target of a seven-million dollar contract placed on him by Santino. Wick then sets out to seek revenge and kill Santino."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [16]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [17]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [18]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, it seems that people generally liked John Wick. The reviews praised the slickness of Keanu Reeves' performance, the brilliance of the action sequences, and the overall fun and entertainment value of the movie. The majority of reviewers highly recommended John Wick, with some even calling it the best action film they've seen in recent years."

In [33]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review URL: /review/rw4854296/?ref_=tt_urv'

In [34]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, a former hitman, is forced back into the world of assassination when an Italian baddie calls in a favor, and John Wick has no choice but to accept. The plot involves John Wick being tasked with killing the Italian baddie's sister in Rome, leading to a contract being put on him, attracting professional killers from all over."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [19]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [20]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/qk/rnr0wf6j3xj452rpryymzb200000gs/T/ipykernel_67099/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [21]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [22]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [23]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [42]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the context provided, opinions on John Wick seem to be divided. Some people really enjoy the movie and find it to be a wild ride, while others think it is horrible and full of nonsense. So, it seems that not everyone generally likes John Wick.'

In [43]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. The URL to that review is: /review/rw4854296/?ref_=tt_urv'

In [44]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the protagonist, played by Keanu Reeves, is a retired assassin who comes out of retirement seeking revenge after his dog is killed and his car is stolen. He goes on a violent rampage to settle old debts and faces off against numerous assassins in locations like Italy, Canada, and Manhattan.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [24]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [25]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [47]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. Reviews praised Keanu Reeves' performance, the action sequences, and the overall entertainment value of the film. The majority of reviews expressed positive sentiments towards John Wick, indicating that it was well-received by the viewers."

In [48]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'"

In [49]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The story is filled with violent action, shootouts, and breathtaking fights as John Wick goes on a relentless vendetta against those who wronged him.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

In [26]:
!pip install -qU langchain_experimental langchain_core==0.2.40

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [27]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [28]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [29]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [30]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [31]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [59]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'People generally liked John Wick based on the positive reviews and high ratings provided in the context.'

In [60]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [61]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is about a retired assassin seeking revenge on the people who killed his dog and took something he loved from him. It involves a lot of action, stylish stunts, and a relatable hero.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

### 1. Create a golden dataset ###
I created the synthetic target data using the notebook gen_synthetic_data.ipynb in the same git directory as this notebook. I used RAGAS SDG, generated 30 questions, and saved it as a file called "sdg_test_data.csv". (I happened to have a handy-dandy notebook from the midterm assignment sitting around, which makes this easy!)

### 2. Evaluate each retriever with retriever-specific RAGAS Metrics ###

For metrics, let's look at *context precision*, *context recall*, and *answer relevancy*. These three will help us understand whether the right answers were ranked first, whether the context covered all aspects of the question, and whether it helped us answer the question. There are other fancy custom metrics we could define, like semantic coverage, recall@k, or others, but let's go with the built in stuff for now.

In [32]:
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall
)
RAGAS_METRICS = [ answer_relevancy, context_precision, context_recall ]

In [43]:
import pandas as pd
from tqdm.asyncio import tqdm_asyncio
from ragas import evaluate
from datasets import Dataset
import time

TEST_DATASET_FILE = "sdg_test_data.csv"

# Utility function to generate responses, run an eval, and save the result
# We could make this more efficient by not re-loading the test data every time
async def gen_rag_responses_and_evaluate(rag_chain,output_filename) -> Dataset:
    """Wrapper function to run a RAG chain against a test dataset and generate/store responses"""
    test_df = pd.read_csv(TEST_DATASET_FILE)

    test_questions = test_df["question"].to_list()
    test_gt = test_df["ground_truth"].to_list()
    print("read test questions")

    answers = []
    contexts = []

    print("generating responses")
    start_time = time.time()

    for question in tqdm_asyncio(test_questions,desc="Processing Questions"):
        response = await rag_chain.ainvoke({"question" : question})
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])
        
    print("Elapsed time to generate answers:",time.time()-start_time,"seconds")

    # Put in huggingface dataset format and save it for later re-use
    response_dataset = Dataset.from_dict({
        "question" : test_questions,
        "answer" : answers,
        "contexts" : contexts,
        "ground_truth" : test_gt
    })

    hf_results = evaluate(response_dataset, 
                   RAGAS_METRICS, 
                   llm=ChatOpenAI())

    # Check out the results, save them to disk
    print(hf_results)

    results_df = pd.DataFrame([hf_results])
    results_df.to_csv(output_filename, index=False)
    return results_df

In [42]:
contextual_compression_results = await gen_rag_responses_and_evaluate(contextual_compression_retrieval_chain,"contextual_compression_resuls.csv")

read test questions
generating responses


Processing Questions: 100%|██████████| 29/29 [00:53<00:00,  1.83s/it]


Elapsed time to generate answers: -53.09505295753479 seconds


Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

{'answer_relevancy': 0.9370, 'context_precision': 0.7529, 'context_recall': 0.8621}


In [44]:
naive_results = await gen_rag_responses_and_evaluate(naive_retrieval_chain,"naive_retrieval_results.csv")

read test questions
generating responses


Processing Questions: 100%|██████████| 29/29 [00:37<00:00,  1.29s/it]


Elapsed time to generate answers: 37.37549805641174 seconds


Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

{'answer_relevancy': 0.8661, 'context_precision': 0.6389, 'context_recall': 0.7069}


In [45]:
multi_query_results = await gen_rag_responses_and_evaluate(multi_query_retrieval_chain,"multi_query_results.csv")

read test questions
generating responses


Processing Questions: 100%|██████████| 29/29 [01:38<00:00,  3.40s/it]


Elapsed time to generate answers: 98.71108818054199 seconds


Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

{'answer_relevancy': 0.8413, 'context_precision': 0.6920, 'context_recall': 0.7644}


In [46]:
parent_document_results = await gen_rag_responses_and_evaluate(parent_document_retrieval_chain,"parent_doc_results.csv")

read test questions
generating responses


Processing Questions: 100%|██████████| 29/29 [00:43<00:00,  1.49s/it]


Elapsed time to generate answers: 43.22631883621216 seconds


Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

{'answer_relevancy': 0.8670, 'context_precision': 0.7443, 'context_recall': 0.7011}


In [49]:
ensemble_results = await gen_rag_responses_and_evaluate(ensemble_retrieval_chain,"ensemble_results.csv")

read test questions
generating responses


Processing Questions: 100%|██████████| 29/29 [01:47<00:00,  3.71s/it]


Elapsed time to generate answers: 107.63798379898071 seconds


Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

{'answer_relevancy': 0.9423, 'context_precision': 0.6771, 'context_recall': 0.8793}


In [48]:
naive_semantic_results = await gen_rag_responses_and_evaluate(semantic_retrieval_chain,"semantic_results.csv")

read test questions
generating responses


Processing Questions: 100%|██████████| 29/29 [00:47<00:00,  1.64s/it]


Elapsed time to generate answers: 47.702778816223145 seconds


Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

{'answer_relevancy': 0.8743, 'context_precision': 0.6807, 'context_recall': 0.6207}


### 3. Compile these in a list and write a small paragraph about which is best for this particular data and why ###

| Retrieval Technique       | Answer Relevancy  | Context Precision | Context Recall    | Total Time    | Cost  |
|---------------------------|-------------------|-------------------|-------------------|---------------|-------|
| Naive                     | 86.6%             | 63.9%             | 70.7%             | 37.4s         | $     |                     
| Naive + Semantic Chunking | 87.4%             | 68.1%             | 62.1%             | 47.7s         | $     |
| Contextual Compression    | 93.7%             | **75.3%**         | **86.2%**         | 53.1s         | $$    |          
| Multi Query               | 91.1%             | 68.6%             | 80.5%             | 98.7s         | $$    |
| Parent Document           | 86.7%             | 74.4%             | 70.7%             | 43.2s         | $$    |
| Ensemble                  | **94.2%**         | 67.8%             | 87.9%             | 108s          | $$$   |

For this dataset, the ensemble technique achieves the highest metrics for answer relevancy -- out of the three we are using here, this is likely the most important one from a user standpoint. However, the Cohere reranker (contextual compression) outperforms in context precision and context recall, at a much lower time and cost compared to the ensemble, so this is likely the best technique for this use case. We have small documents and must combine many to answer some of the questions, so it makes sense that contextual compression with reranking would be a high performing approach.

If we were to try an additional run, I'd try combining contextual compression with semantic chunking. 

Note that I didn't measure costs exactly, though it is possible to do in Langsmith if that were important. Instead, I've included a comparative representation of cost in the table, with the naive techniques being lowest cost, the techniques that generate extra LLM calls (multi query) or external paid services (Cohere) in the middle, and the ensemble technique which combines the expenses of all other techniques as the most expensive.